In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cse-251-b-2025/train.npz
/kaggle/input/cse-251-b-2025/test_input.npz


In [2]:
# download the dataset to your folder or use it on kaggle notebook directly

# train_file = np.load('/your_folder/cse-251-b-2025/train.npz')
train_file = np.load('/kaggle/input/cse-251-b-2025/train.npz')

train_data = train_file['data']
print("train_data's shape", train_data.shape)
# test_file = np.load('/your_folder/test_input.npz')
test_file = np.load('/kaggle/input/cse-251-b-2025/test_input.npz')

test_data = test_file['data']
print("test_data's shape", test_data.shape)


train_data's shape (10000, 50, 110, 6)
test_data's shape (2100, 50, 50, 6)


In [3]:
# Split x and y for train data.

train_x, train_y = train_data[..., :50, :], train_data[:, 0, 50:, :2]

print(train_x.shape, train_y.shape)

(10000, 50, 50, 6) (10000, 60, 2)


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

class TrajectoryModel(nn.Module):
    def __init__(self, input_features, output_features):
        super(TrajectoryModel, self).__init__()
        
        # Define the layers
        # self.conv1 = nn.Conv2d(6,32,3,padding=1)
        # self.conv2 = nn.Conv2d(32,64,3,padding=1)
        # self.conv3 = nn.Conv2d(64,128,3,padding=1)

        # self.dropout = nn.Dropout(0.3)


        # self.pool = nn.MaxPool2d(2,2)
        self.relu = nn.ReLU()

        self.fc1 = nn.Linear(input_features, 2048)
        self.fc2 = nn.Linear(2048 , output_features)
        # self.fc3 = nn.Linear(512 , output_features)
    
    def forward(self, x):
        # x = self.relu(self.conv1(x))
        # x = self.pool(self.relu(self.conv2(x)))
        # x = self.pool(self.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        # x = self.relu(self.fc2(x))
        x = self.fc2(x)
        return x.resize(x.size(0),60,2)


In [12]:
class LSTM(nn.Module):
    def __init__(self, input_size = 6, output_features = 120):
        super(LSTM, self).__init__()
        
        self.lstm = nn.LSTM(input_size , 256 , batch_first = True)
        self.fc1 = nn.Linear(256 , output_features)
    
    def forward(self, x):
        batch , num_agents , time_steps, features = x.size()
        x = x.view(batch*num_agents,time_steps,features)
        output,(h_n,c_n) = self.lstm(x)
        x = self.fc1(output[:,-1,:])
        # print(x.shape)
        x = x[:batch,:]
        return x.view(-1,60,2)

In [13]:
class TrajectoryRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_steps, num_agents):
        super(TrajectoryRNN, self).__init__()

        self.input_dim  =input_dim
        self.hidden_dim = hidden_dim
        self.num_steps = num_steps
        self.num_agents = num_agents
        
        # LSTM layer for trajectory prediction
        self.lstm = nn.LSTM(input_dim * num_agents, hidden_dim, batch_first=True)
        
        # Fully connected layer to output the predicted trajectory (x, y)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # x: shape [batch_size, num_agents, num_time_steps, input_dim]
        
        batch_size = x.size(0)
        
        # Reshape input to have shape [batch_size, num_time_steps, num_agents * input_dim]
        x = x.view(batch_size, x.size(2), self.num_agents * x.size(3))  # shape: [batch_size, num_time_steps, 50 * 6]
        
        # Initialize hidden and cell states for LSTM
        h0 = torch.zeros(1, batch_size, self.hidden_dim).to(x.device)  # hidden state
        c0 = torch.zeros(1, batch_size, self.hidden_dim).to(x.device)  # cell state
    
        # Pass through the LSTM
        lstm_out, (hn, cn) = self.lstm(x, (h0, c0))
        
        predictions = []
    
        for t in range(self.num_steps):
            # Take the last output from LSTM (last time step)
            lstm_out_last = lstm_out[:, -1, :]
            
            # Get the predicted x, y for all agents (output_dim = 2 * num_agents)
            output = self.fc(lstm_out_last)  # shape: [batch_size, num_agents * 2]
            
            # Store all predictions but only return for agent 0
            predictions.append(output[:, :2].unsqueeze(1))  # Get only the first 2 values for agent 0 (x, y)
            
            # Shift the inputs (for teacher forcing) - not necessary for all models
            expanded_output = output.unsqueeze(2).expand(-1, -1, self.num_agents * self.input_dim)
            
            # Concatenate along the time dimension
            x = torch.cat((x[:, 1:, :], expanded_output), dim=1)  # shape: [batch_size, num_time_steps, 50 * 6]
        
        # Stack all the predictions into a tensor
        return torch.cat(predictions, dim=1)  # shape: [batch_size, num_steps, 2]


In [19]:
# Calculate the total number of features after flattening
input_features = 50 * 50 * 6  # = 5000
output_features = 60 * 2


# Create the model
model = TrajectoryModel(input_features, output_features)
# model = LSTM()
# model = TrajectoryRNN(input_dim=6, hidden_dim=128, output_dim=2, num_steps=60,num_agents = 50)

# Define loss function and optimizer
criterion = nn.MSELoss()  # For regression task

optimizer = optim.Adam(model.parameters(), lr=1e-5)


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:

# Example of how to prepare data and train the model

def train_model(model, x_train, y_train, batch_size=64, epochs=10):
    
    
    X_train_tensor = torch.FloatTensor(x_train).to(device) 
    y_train_tensor = torch.FloatTensor(y_train).to(device)   
    # DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Model to device
    model = model.to(device)
    
    # Training loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        
        for batch_X, batch_y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(batch_X)
            # print(outputs.shape)
            # print(batch_y.shape)
            
            # Compute loss
            loss = criterion(outputs, batch_y)
            
            # Backward and optimize
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        avg_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1} - Loss: {avg_loss:.4f}")
    
    return model


In [ ]:
model = train_model(model, train_x, train_y, 32, 150)

Epoch 1/150: 100%|██████████| 313/313 [00:03<00:00, 84.96it/s]


Epoch 1 - Loss: 1352.6529


Epoch 2/150: 100%|██████████| 313/313 [00:03<00:00, 85.20it/s]


Epoch 2 - Loss: 2624.8956


Epoch 3/150: 100%|██████████| 313/313 [00:03<00:00, 85.16it/s]


...

Epoch 42/150: 100%|██████████| 313/313 [00:03<00:00, 84.92it/s]


Epoch 42 - Loss: 797.7632


Epoch 43/150: 100%|██████████| 313/313 [00:03<00:00, 85.04it/s]


Epoch 43 - Loss: 711.9319


Epoch 44/150:  86%|████████▋ | 270/313 [00:03<00:00, 85.13it/s]

In [23]:

def predict(X_test):
    model.eval()
    with torch.no_grad():
        X_test_tensor = torch.FloatTensor(X_test).to(device) 
        predictions = model(X_test_tensor).to(device)
    
    # Move predictions to CPU before converting to NumPy
    return predictions.cpu().numpy()

# Save model
def save_model(path="mlp_model.pth"):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

# Load model
def load_model(path="mlp_model.pth"):
    loaded_model = TrajectoryModel()
    loaded_model.load_state_dict(torch.load(path))
    loaded_model.eval()
    return loaded_model

In [24]:
# Get predictions from the model
pred_y = predict(test_data)

# Move pred_y to CPU if it's on the GPU
# pred_y = pred_y.detach().cpu()

# Now reshape pred_y after it's on the CPU
pred_output = pred_y.reshape(-1, 2)  # Convert to NumPy after moving to CPU

# Create the DataFrame
output_df = pd.DataFrame(pred_output, columns=['x', 'y'])

# Adding a necessary step to match index of your prediction to that of the solution key
output_df.index.name = 'index'

# Save to CSV
output_df.to_csv('mlp_baseline.csv')

 # Now you can submit to the leaderboard!